In [10]:
import numpy as np
import pandas
#df2 = pandas.read_csv('test.csv', encoding='latin-1')

In [11]:
df = pandas.read_csv('train.csv', encoding='latin-1', names=["Sentiment", "ID", "Date", "Query","Username","Text"])
df = df.drop(["ID", "Date", "Query","Username"], axis=1)

In [12]:
df['Text'] = df['Text'].str.replace("&gt;_&lt;", "sad")
df['Text'] = df['Text'].str.replace(";-;", "sad")
df['Text'] = df['Text'].str.replace(";_;", "sad")
df['Text'] = df['Text'].str.replace(" D: ", " sad ")
df['Text'] = df['Text'].str.replace(" :D ", " happy ")
df['Text'] = df['Text'].str.replace(" ;D", " happy")
df['Text'] = df['Text'].str.replace("xd", "laugh")
df['Text'] = df['Text'].str.replace("=.=", "confused")
df['Text'] = df['Text'].str.replace("o_o", "confused")
df['Text'] = df['Text'].str.replace("O_o", "confused")

df['Text'] = df['Text'].str.lower()

#df['Text'] = df['Text'].str.replace("...", " ")
df["Text"] = df["Text"].str.replace('(\@\w+.*?)',"")
df["Text"] = df["Text"].str.replace('(\#\w+.*?)',"")
df['Text'] = df['Text'].str.replace('http[^\s]*',"")
df['Text'] = df['Text'].str.replace(".", "")
df['Text'] = df['Text'].str.replace(",", "")
df['Text'] = df['Text'].str.replace("-", "")
df['Text'] = df['Text'].str.replace("`", "")
df['Text'] = df['Text'].str.replace("'", "")
df['Text'] = df['Text'].str.replace(": ", " ")
df['Text'] = df['Text'].str.replace("; ", " ")
df['Text'] = df['Text'].str.replace("&amp;", "and")
df['Text'] = df['Text'].str.replace("&quot;", "and")
df['Text'] = df['Text'].str.replace("?", "")
df['Text'] = df['Text'].str.replace("!", "")
df['Text'] = df['Text'].str.replace("*", "")
df["Text"] = df["Text"].str.replace('ï¿½',"")
df['Text'] = df['Text'].str.replace("&amp", "")
df['Text'] = df['Text'].str.replace("&quot", "")
df['Text'] = df['Text'].str.replace("&gt", "")
df['Text'] = df['Text'].str.replace("&lt", "")
df['Text'] = df['Text'].str.replace("  ", " ")
df['Text'] = df['Text'].str.replace("  ", " ")
df['Text'] = df['Text'].str.replace("  ", " ")

df['Text'] = df['Text'].str.replace(" @ ", " ")
df['Text'] = df['Text'].str.replace("upits", "up its")
df['Text'] = df['Text'].str.replace("Of to ", "off to ")
df['Text'] = df['Text'].str.replace("brightside", "bright side")
df['Text'] = df['Text'].str.replace("thaanks", "thanks")
df['Text'] = df['Text'].str.replace("thaaanks", "thanks")

df['Text'] = df['Text'].str.lstrip(' ')
df['Text'] = df['Text'].str.rstrip(' ')



C:\Users\hyun9\AppData\Local\Temp\ipykernel_28784\883629432.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text'] = df['Text'].str.replace("=.=", "confused")
C:\Users\hyun9\AppData\Local\Temp\ipykernel_28784\883629432.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Text"] = df["Text"].str.replace('(\@\w+.*?)',"")
C:\Users\hyun9\AppData\Local\Temp\ipykernel_28784\883629432.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Text"] = df["Text"].str.replace('(\#\w+.*?)',"")
C:\Users\hyun9\AppData\Local\Temp\ipykernel_28784\883629432.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text'] = df['Text'].str.replace('http[^\s]*',"")
C:\Users\hyun9\AppData\Local\Temp\ipykernel_28784\883629432.py:18: FutureWarning: The default value of regex will change from True to F

In [13]:
print(df["Text"][2])
print(df["Text"][20])

i dived many times for the ball managed to save 50% the rest go out of bounds
i was out most of the day so didnt get much done


In [14]:
# Dataset

import torch
import numpy as np
from transformers import BertTokenizer

MAXLENGTH = 512


tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels for labels in df['Sentiment']]
        self.texts = [tokenizer(text, padding='max_length', max_length = MAXLENGTH, truncation=True, return_tensors="pt") for text in df['Text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [15]:
np.random.seed(142)
trainX, evaluationX, testX = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])

In [16]:
# Reduce Dataset Size (Temp)

trainX = trainX.drop(trainX.index[0:1255000])
testX = testX.drop(testX.index[0:159900])
evaluationX = evaluationX.drop(evaluationX.index[0:159900])

In [17]:
trainX.shape, testX.shape, evaluationX.shape

((25000, 2), (100, 2), (100, 2))

In [18]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [19]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

EPOCHS = 5 # ---------------
model = BertClassifier()
LR = 1e-6

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
aa = torch.utils.data.DataLoader(a, batch_size=2, shuffle=True)

In [33]:
aa.dataset.labels

[0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 4,
 4,
 0,
 0,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 4,
 0,
 0,
 4,
 4,
 0,
 4,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 0,
 4,
 0,
 0,
 4,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 0,
 4,
 0,
 4,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 4,
 0,
 0,
 0,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 4,
 0,
 4,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 4,
 4,
 4,
 0,
 0,
 4,
 0,
 4,
 4,
 4,
 4,
 0,
 4,
 0,
 4,
 0,
 4,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 4,
 0,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 0,
 4,
 0,


In [ ]:
train(model, trainX, evaluationX, LR, EPOCHS)

In [2]:
from transformers import BertModel, BertTokenizer, BertConfig
import torch

enc = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenizing input text
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = enc.tokenize(text)

# Masking one of the input tokens
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
indexed_tokens = enc.convert_tokens_to_ids(tokenized_text)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Creating a dummy input
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
dummy_input = [tokens_tensor, segments_tensors]

# Initializing the model with the torchscript flag
# Flag set to True even though it is not necessary as this model does not have an LM Head.
config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
    num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072, torchscript=True)

# Instantiating the model
model = BertModel(config)

# The model needs to be in evaluation mode
model.eval()

# If you are instantiating the model with `from_pretrained` you can also easily set the TorchScript flag
model = BertModel.from_pretrained("bert-base-uncased", torchscript=True)

# Creating the trace
traced_model = torch.jit.trace(model, [tokens_tensor, segments_tensors])
torch.jit.save(traced_model, "traced_bert.pt")

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

C:\Users\hyun9\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hyun9\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.b

In [3]:
tokens_tensor

tensor([[  101,  2040,  2001,  3958, 27227,  1029,   102,  3958,   103,  2001,
          1037, 13997, 11510,   102]])

In [4]:
segments_tensors

tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])